# Functional Class and Curvature Analysis
This notebook illustrates the complete analysis example using Here Map Content (HMC) attributes.

### Dependencies
* Catalogs: [rib-2](https://platform.here.com/data/hrn:here:data::olp-here:rib-2)
* Layers: [Topology Geometry](https://platform.here.com/data/hrn:here:data::olp-here:rib-2/topology-geometry), [Road Attributes](https://platform.here.com/data/hrn:here:data::olp-here:rib-2/road-attributes), [ADAS Attributes](https://platform.here.com/data/hrn:here:data::olp-here:rib-2/adas-attributes)

### Workflow
* Start with a bounding box that contains portions of two tiles
* Extract geometry for the two tiles
* Keep only the segments that are (at least partially) in original bounding box
* Extract Functional Class (FC) attributes for those segments
* Select segments with FC = 3, 4, and 5
* Extract curvature for the survived segments
* Analyze average curvature
* Drop segments below the average curvature

## Set API key (here.access.key.secret)

In [7]:
import os
os.environ["LS_API_KEY"] = "Cbf74Up9FOMkD3W2o1OsVs8sisG3CBmjz210SFhIo9c"

## Get partition ID's from a given bounding box

In [8]:
from here.geotiles.heretile import in_bounding_box
from shapely.geometry import Point

zoom_level = 12
west = 13.41
east = 13.49
south = 52.50
north = 52.54
center = Point(west, south)
tileIds = list(in_bounding_box(west=west, south=south, east=east, north=north, level=zoom_level))

# partition ID's: string representation of tile IDs - this will be used for extracting attributes later
partition_ids = [str(tile) for tile in tileIds]
print("partition ids:", partition_ids)

partition ids: ['23618402', '23618403']


## Visualize the bounding box and tile boundaries
- bounding box: Red
- tile boundaries: Gray

In [9]:
from here.inspector import inspect
from here.inspector.styles import Color
from shapely.geometry import Polygon

# construct a bounding box geojson
bb_polygon = Polygon([(west, south), (west, north), (east, north), (east, south), (west, south)])

# create inspect object with tile boundaries and the bounding box
inspect(layers={"bounding_box": [bb_polygon]},
        tiles=tileIds,
        center=center,
        zoom=11,
        layers_style={"bounding_box": Color.RED},
        tiles_style=Color.GRAY)

AttributeError: 'Map' object has no attribute 'add'

## Extract topology geometry and clip by bounding box

In [10]:
from here.platform import Platform
from here.content.hmc2.hmc import HMC
from here.geopandas_adapter import GeoPandasAdapter
import pandas as pd
import geopandas as gpd

# create HMC object
hmc = HMC(Platform(), adapter=GeoPandasAdapter())

ValueError: Unknown layer id here-places

In [ ]:
# get segments dataframe from given tiles
segment_df = hmc.topology_geometry.get(tileIds, "segment")
segment_df

### Clip segments by the bounding box
Since the geometry objects for the segment dataframe is LineString, the intersection method will do the clipping. See the documentation of shapely: https://shapely.readthedocs.io/en/stable/manual.html

In [ ]:
clipped_segments = segment_df.intersection(bb_polygon)
segment_in_bb = segment_df[clipped_segments.apply(lambda x: x.length != 0)]
segment_in_bb

In [ ]:
# show the clipped segments and original tile boundaries
inspect(features=segment_in_bb["geometry"], tiles=tileIds, tiles_style=Color.GRAY)

## Get functional class attribute with segment reference

In [ ]:
# get functional_class attributes
fc_df_seg = hmc.road_attributes.get_referencing(tileIds, "functional_class", "segment")
fc_df_seg

## Select FC = 3, 4, or 5

In [ ]:
# select FC = 3, 4, 5
fc345_df = fc_df_seg[
    (fc_df_seg["attribute.functional_class"] == "FUNCTIONAL_CLASS_3") |
    (fc_df_seg["attribute.functional_class"] == "FUNCTIONAL_CLASS_4") |
    (fc_df_seg["attribute.functional_class"] == "FUNCTIONAL_CLASS_5")
]
fc345_df

In [ ]:
# change indexes into actual columns
fc345_df = fc345_df.reset_index()
# rename columns for easier merging later on
fc345_df.rename(columns={'ref_partition': 'partition_id'}, inplace=True)
fc345_df.rename(columns={'ref_identifier': 'segment_id'}, inplace=True)
fc345_df

In [ ]:
# join two dataframe by partitionId, segmentId
segment_fc345_df = segment_in_bb.merge(fc345_df,
                                       how="inner",
                                       on=["partition_id", "segment_id"])
segment_fc345_df

## Show the segments with FC = 3, 4, or 5

In [ ]:
inspect(segment_fc345_df["geometry"])

## Show different colors for each FC

In [ ]:
fcs = {}
for fc, gdf in segment_fc345_df.groupby("attribute.functional_class"):
    fcs[fc] = gdf["geometry"]
inspect(layers=fcs)

## Extract curvatures

In [ ]:
# get segment curvatures (curvatures at the nodes are not used in this notebook example)
segment_curvature_df = hmc.adas_attributes.get(partition_ids, "curvature_heading")
segment_curvature_df

## Select segments with high curvatures

In [ ]:
# create a column called abs_curvature to calculate average curvature
segment_curvature_df["abs_curvature"] = segment_curvature_df.apply(lambda x: abs(x["curvature"]), axis=1)
average_curvature = segment_curvature_df["abs_curvature"].mean()

# select segments with curvature higher than average
highcurv_segments = segment_curvature_df[segment_curvature_df["abs_curvature"] > average_curvature]

# extract segment_id from segment anchors
highcurv_segments = highcurv_segments.copy()
highcurv_segments["segment_id"] = highcurv_segments.apply(
    lambda x: x["segment_anchors"][0].oriented_segment_ref[0][0].segment_id,
    axis=1
)

highcurv_segments

## Join two dataframes
Inner join between the dataframe with FC=3,4,5 and the dataframe with high curvatures

In [ ]:
segment_fc345_highcurv_df = segment_fc345_df.merge(highcurv_segments,
                                                   how="inner",
                                                   on=["partition_id", "segment_id"])
segment_fc345_highcurv_df

In [ ]:
# drop abs_curvature which was used to calculate average curvature
merged_df = segment_fc345_highcurv_df.drop("abs_curvature", axis=1)
merged_df.columns

## Show the survived segments

In [ ]:
inspect(merged_df["geometry"])

<span style="float:left; margin-top:3px;"><img src="https://www.here.com/themes/custom/here_base_theme_v2/logo.svg" alt="HERE Logo" height="60" width="60"></span><span style="float:right; width:90%;"><sub><b>Copyright (c) 2020-2021 HERE Global B.V. and its affiliate(s). All rights reserved.</b>
This software, including documentation, is protected by copyright controlled by HERE. All rights are reserved. Copying, including reproducing, storing, adapting or translating, any or all of this material requires the prior written consent of HERE. This material also contains confidential information which may not be disclosed to others without the prior written consent of HERE.</sub></span>